In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

# 步骤1：收集数据

In [5]:
train_pd = pd.read_csv('./train.csv')
test_pd = pd.read_csv('./test.csv')

In [6]:
# 数据总体信息概览
train_pd.info()
print("="*50)
test_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 596 entries, 0 to 595
Data columns (total 12 columns):
PassengerId    596 non-null int64
Survived       596 non-null int64
Pclass         596 non-null int64
Name           596 non-null object
Sex            596 non-null object
Age            472 non-null float64
SibSp          596 non-null int64
Parch          596 non-null int64
Ticket         596 non-null object
Fare           596 non-null float64
Cabin          136 non-null object
Embarked       594 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 56.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float

# 步骤2：数据准备

In [21]:
X_pd = train_pd[["Pclass", 'Sex', 'SibSp', 'Parch', 'Fare']]
y_pd = train_pd[['Survived']]

In [22]:
X_pd['Sex'] = X_pd['Sex'].map({"female":1,"male":0}).astype(int)
X_pd.head()

/Users/chenzu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Pclass,Sex,SibSp,Parch,Fare
0,3,0,1,0,7.2500
1,1,1,1,0,71.2833
2,3,1,0,0,7.9250
3,1,1,1,0,53.1000
4,3,0,0,0,8.0500


In [26]:
train_X,val_X,train_y,val_y = train_test_split(
    X_pd.values,y_pd.values,test_size=0.1,random_state=0)

# 步骤3：选择一个模型

In [27]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()

# 步骤4：训练

In [29]:
clf.fit(train_X, train_y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

# 步骤5：评估

In [30]:
acc = clf.score(train_X, train_y)
print('训练集准确率：'+str(acc))

acc = clf.score(val_X, val_y)
print('验证集准确率：'+str(acc))

训练集准确率：0.9216417910447762
验证集准确率：0.7833333333333333


# 步骤7：预测

In [47]:
# 测试集特征预处理
test_X_pd = test_pd[["Pclass", 'Sex', 'SibSp', 'Parch', 'Fare']]
test_X_pd['Sex'] = test_X_pd['Sex'].map({"female":1,"male":0}).astype(int)

# 测试集缺失值处理
test_X_pd[test_X_pd['Fare'].isnull().values == True]
print(test_X_pd.loc[152])
test_X_pd['Fare'].fillna(test_X_pd['Fare'].mean(), inplace=True)
test_X_pd.loc[152]

Pclass    3.0
Sex       0.0
SibSp     0.0
Parch     0.0
Fare      NaN
Name: 152, dtype: float64


/Users/chenzu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


Pclass     3.000000
Sex        0.000000
SibSp      0.000000
Parch      0.000000
Fare      35.627188
Name: 152, dtype: float64

In [48]:
# 测试集预测
test_y = clf.predict(test_X_pd.values)

In [51]:
# 乘客id和预测结果拼接
submission = pd.DataFrame({
        "PassengerId": test_pd["PassengerId"],
        "Survived": test_y
    })
submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [52]:
# 保存cvs文件
submission.to_csv('submission_decesiontree'
                  +".csv", 
                  index=False)